# Dazbo's YouTube Demos

## Overview

Examples of how to work with YouTube using Python.

A few useful notes:

- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/dazbo-python-demos" target="_blank">Dazbo-Python-Demos</a>.
- Check out further guidance - including tips on how to run the notebook, in the project's `README.md`.
- For example, you could...
  - Run the notebook locally, in your own Jupyter environment.
  - Run the notebook in a cloud-based Jupyter environment, with no setup required on your part!  For example, <a href="https://colab.research.google.com/github/derailed-dash/dazbo-python-demos/blob/main/notebooks/youtube-demos.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a>
- **To run the notebook, first execute the cells in the [Setup](#Setup) section, as described below.** Then you can experiment with any of the subsequent cells.

## Setup

First, let's install any dependent packages:

In [ ]:
%pip install --upgrade --no-cache-dir dazbo-commons pytubefix yt_dlp

In [5]:
import logging
import re
from pathlib import Path
from dataclasses import dataclass
import dazbo_commons as dc

Now we'll setup logging. Here I'm using coloured logging from my [dazbo-commons](https://pypi.org/project/dazbo-commons/) package. Feel free to change the logging level.

In [ ]:
# Setup logging
APP_NAME="dazbo-yt-demos"
logger = dc.retrieve_console_logger(APP_NAME)
logger.setLevel(logging.DEBUG)
logger.info("Logger initialised.")
logger.debug("DEBUG level logging enabled.")

Here we initialise some file path locations, e.g. an output folder.

In [ ]:
locations = dc.get_locations(APP_NAME)
for attribute, value in vars(locations).items():
    logger.debug(f"{attribute}: {value}")

Now some utility functions.

In [8]:
def clean_filename(filename):
    """ Create a clean filename by removing unallowed characters. """
    pattern = r'[^a-zA-Z0-9._\s-]'
    return  re.sub(pattern, '_', filename)

## Downloading Videos and Extracting Audio

In [9]:
# YouTube videos to download
urls = [
    "https://www.youtube.com/watch?v=udRAIF6MOm8",  # Sigrid - Burning Bridges
    "bla", # Test a bad URL
    "https://www.youtube.com/watch?v=CiTn4j7gVvY",  # Melissa Hollick - I Believe
    "https://www.youtube.com/watch?v=d4N82wPpdg8",  # Jerry Heil & Alyona Alyona - Teresa & Maria
]

### With PyTubeFix

Here I'll use the [pytubefix](https://github.com/JuanBindez/pytubefix) library to download YouTube videos, and then to download mp3 audio-only streams as files.

This library is a community-maintained fork of `pytube`. It was created to provide quick fixes for issues that the official pytube library faced, particularly when YouTube's updates break `pytube`.

Pros:

- The library is very easy to use.
- We can work with video, audio, channels, playlists, and even search and filter.
- It is [well documented](https://pytubefix.readthedocs.io/en/latest/).
- It can be used from the command line, with its simple CLI.
- It is VERY FAST!

Cons:

- Does not offer some of the more sophisticated capabilities that are offered by `yt_dlp`.
- It does not appear to set mp3 headers correctly, meaning that subsequent programmatic maninpulation (e.g. converting to wav) is not as trivial as it could be!

In [ ]:

from pytubefix import YouTube
from pytubefix.cli import on_progress

videos = []
audios = []

for i, url in enumerate(urls):
    logger.info(f"Downloads progress: {i+1}/{len(urls)}")

    try:
        yt = YouTube(url, on_progress_callback=on_progress)
        logger.info(f"Getting: {yt.title}")
        video_stream = yt.streams.get_highest_resolution()
        if not video_stream:
            raise Exception("Stream not available.")
        
        # YouTube resource titles may contain special characters which 
        # can't be used when saving the file. So we need to clean the filename.
        cleaned = clean_filename(yt.title)
        output_locn = f"{locations.output_dir}/pytubefix"
        
        video_output = f"{output_locn}/{cleaned}.mp4"
        logger.info(f"Downloading video {cleaned}.mp4 ...")
        video_stream.download(output_path=output_locn, filename=f"{cleaned}.mp4")
        videos.append(f"{output_locn}/{cleaned}.mp4")
    
        logger.info(f"Creating audio...")
        audio_stream = yt.streams.get_audio_only()
        audio_stream.download(output_path=output_locn, filename=cleaned, mp3=True)
        audios.append(f"{output_locn}/{cleaned}.mp3")
        
        logger.info("Done")
        
    except Exception as e:        
        logger.error(f"Error processing URL '{url}'.")
        logger.error(f"The cause was: {e}") 
        
logger.info(f"Downloads finished.")
for video in videos:
    logger.info(video)
for audio in audios:
    logger.info(audio)


### With YT_DLP

I wanted to try the other popular YouTube package: [yt-dlp](https://pypi.org/project/yt-dlp/). The [repo](https://github.com/yt-dlp/yt-dlp) repo is a fork of the now unmaintained `youtube-dl`. 

Pros:

- It is very powerful, with far more options and features than `pytubefix`.
- It can be installed as a standalone command-line executable, or as a pip-installable Python package.
- Sets mp3 headers properly!

Cons:

- It is more complicated to use.
- The documentation is complex. And there's no real Python-specific documentation.
- It depends on having ffmpeg installed for many use cases.
- It is significantly slower that `pytubefix` for performing video download and audio extraction.


In [ ]:
import yt_dlp

videos = []
audios = []

for i, url in enumerate(urls):
    logger.info(f"Downloads progress: {i+1}/{len(urls)}")

    try:
        # Options for downloading the video
        output_locn = f"{locations.output_dir}/yt_dlp"
        
        video_opts = {
            'format': 'best',  # Download the best quality video
            'outtmpl': f'{output_locn}/%(title)s.%(ext)s',  # Save video in output directory
        }
        
        # Download the video
        with yt_dlp.YoutubeDL(video_opts) as ydl:
            logger.info("Downloading video...")
            ydl.download([url])
        
        # Options for extracting audio and saving as MP3
        audio_opts = {
            'format': 'bestaudio',  # Download the best quality audio
            'outtmpl': f'{output_locn}/%(title)s.%(ext)s',  # Save audio in output directory
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
            }],
        }
        
        # Download and extract audio
        with yt_dlp.YoutubeDL(audio_opts) as ydl:
            logger.info("Extracting and saving audio as MP3...")
            ydl.download([url])
        
    except Exception as e:        
        logger.error(f"Error processing URL '{url}'.")
        logger.error(f"The cause was: {e}") 
        
logger.info(f"Downloads finished. Check out files at {output_locn}.")
for video in videos:
    logger.info(video)
for audio in audios:
    logger.info(audio)

## Audio Conversion

### Install Packages

- [ffmpeg](https://ffmpeg.org/): in order to render video output, i.e. for visualisations.

In [ ]:
import os
import platform

def install_software():
    os_name = platform.system()
    logger.info(f"Installing packages on {os_name}...")
    if os_name == "Windows":
        os.system("winget install ffmpeg --silent --no-upgrade")
    elif os_name == "Linux":
        os.system("apt-get -qq update && apt-get -qq -y install ffmpeg")
    elif os_name == "Darwin":
        os.system("brew install ffmpeg")
    else:
        logger.error(f"Unsupported operating system: {os_name}")

install_software()

logger.info("Note that installed applications may not be immediately available after first installing.\n" \
            "It may be necessary to relaunch the notebook environment.")

Check ffmpeg version:

In [ ]:
!ffmpeg -version

## Transcribing Audio to Text

### Extracting Audio Using Python Speech Recognition

The Python `speech_recognition` package has a number of built in `Recognizer` implementations.Here I'm using the [Google Web Speech API](https://wicg.github.io/speech-api/) `Recognizer`, which has its default API key hard coded into the Python `speech_recognition` library. It is free, but has some limitations. For example, it only allows a max of 60s segments.

It has limited speech recognition capability. Also, it's not going to natively detect language. So our next step will be to add some more smarts!

In [ ]:
%pip install --upgrade --no-cache-dir pydub SpeechRecognition ffmpeg-python

In [59]:
import speech_recognition as sr
from pydub import AudioSegment
import ffmpeg
import traceback
from io import BytesIO

In [ ]:
def divide_chunks(sound, chunksize=60000):
    """ Split audio file into 60s chunks """
    for i in range(0, len(sound), chunksize):
        yield sound[i:i + chunksize]
        
for mp3_file in audios:
    logger.info(f"Converting {mp3_file}...")
    try:
        wav_file = mp3_file.replace(".mp3", ".wav") # The name of our target wav file
        if not os.path.isfile(wav_file):
            # AudioSegment conversion does not work if the mp3 headers are broken
            # sound = AudioSegment.from_mp3(mp3_file)
            # sound.export(wav_file, format="wav")
            
            # Use ffmpeg as a workaround for conversion. However, it's a lot slower
            ffmpeg.input(mp3_file).output(wav_file).run() # Convert with ffmpeg
            logger.info(f"Successfully converted {mp3_file} to {wav_file}.")
        else:
            logger.info(f"Already exists: {wav_file} - not converting.")
        
        wav = AudioSegment.from_wav(wav_file) # Read the audio
        chunks = list(divide_chunks(wav)) # split the wav into chunks
        
        recogniser = sr.Recognizer()
        extracts = {}
        for index, chunk in enumerate(chunks):
            wav_io = BytesIO()
            chunk.export(wav_io, format='wav')
            wav_io.seek(0)  # Move to the start of the BytesIO object before reading from it
                
            with sr.AudioFile(wav_io) as source:
                audio_data = recogniser.record(source)

            try:
                extracted = recogniser.recognize_google(audio_data)
                extracts[index] = extracted
            except sr.UnknownValueError:
                # Log the unknown value error and continue
                logger.warning(f"Chunk {index}: Could not understand the audio. Maybe it was empty.")
            finally:
                wav_io.close() 
        
        for idx, extract in extracts.items():
            logger.info(f"{idx}: {extract}")

    except ffmpeg.Error as e:
        logger.error(f"FFmpeg failed to convert {mp3_file}: {str(e)}")
    except Exception as e:
        logger.error("Unexpected error.", exc_info=e)